## Scrap the weather 
* temperature, humidity, visibility, air pressure, wind speed, datetime data from this website
* https://eg.freemeteo.com/weather/cairo/current-weather/location/?gid=360630&language=english&country=egypt
* for 10 times one read in a minute (so basicly the process takes 10 minutes)
and put them into a CSV & Excel file

#### Import some helper modules 

In [4]:
import os
import csv
import json
import re
from datetime import datetime
import openpyxl
from bs4 import BeautifulSoup
import requests
import time 

#### Function to clean strings 
* take a string as input
* return cleaned string from spaces and lines 

In [6]:
def clean_str (Str : str)->str:
    cleaned = ''
    for i in Str:
        tmp = ''
        for j in i.strip():
            if j not in ['\n','\r']:
                tmp += j
        cleaned += tmp
    return cleaned

#### Save Inof into csv file 
* take a list of lists as input 
* save them into csv file 

In [7]:
def write_csv(*args):
    fn = ['date' , 'time' , 'temp' , 'humidity' , 'wind_speed', 'pressure' , 'visibility']
    m = 'w' if not os.path.exists('weather_scrape.csv') else 'a'
    with open('weather_scrape.csv' , m , newline='')as f:
        writer = csv.DictWriter(f , fieldnames=fn)
        if m == 'w':
            writer.writeheader()
        writer.writerow({k : val for k , val in zip(fn , args)})

#### Save Info in Excel file 
* take list of lists as input
* Save them in excel file 

In [8]:
def write_xl(*args):
    fn = ['date' , 'time' , 'temp' , 'humidity' , 'wind_speed', 'pressure' , 'visibility']
    if not os.path.exists('weather_scrape.xlsx'):
        wb = openpyxl.Workbook()
        sheet = wb.active
        row , col = sheet.max_row , sheet.max_column
        for i in fn:
            sheet.cell(row , col ).value = i
            col += 1
        wb.save('weather_scrape.xlsx')
    wb = openpyxl.load_workbook('weather_scrape.xlsx')
    sheet = wb.active
    row , col = sheet.max_row + 1  , 1
    for info in args:
        sheet.cell(row , col).value = info
        col += 1
    wb.save('weather_scrape.xlsx')

#### Scrapping Process
* make repuest to web bage by requests 
* parse the request to html using BeautifulSoup and get the body 
* get weather info 10 times one minutes for each and save into csv & excel file 

In [8]:
for i in range (10):
    req = requests.get('https://eg.freemeteo.com/weather/cairo/current-weather/location/?gid=360630&language=english&country=egypt')
    soup = BeautifulSoup(req.content , 'html.parser')
    body = soup.body
    Weather_Today = body.find('div', {'class':"weather-now"})
    Weather_Now = Weather_Today.find('div' , {'class':"hourly-www-update"})
    date_time = Weather_Now.find('span' , {'class':"wicon"})['data-date'].split('T')
    date , Time = date_time[0] , date_time[1][:-1]
    temp = Weather_Now.find('span' , {'class':"temp"}).text
    humidity = Weather_Now.find('span' , {'class':"humid"}).text
    wind_speed = Weather_Now.find('span' , {'class':"winds"}).text
    chart = Weather_Today.find('div' , {'class':"chart"})
    last_renew_info = chart.find('div' , {'class':"last-renew-info"})
    stats = last_renew_info.find('div' , {'class':"stats"})
    stats = clean_str(stats.text)
    stats = stats.split('|')
    Visibility , Pressure = stats[1] , stats[2]
    Visibility = Visibility.split(':')[1][:-1]
    Pressure = Pressure.split('mb')[0].split(':')[1]
    write_csv(date , Time , temp , humidity , wind_speed, Pressure , Visibility)
    write_xl(date , Time , temp , humidity , wind_speed, Pressure , Visibility)
    time.sleep(60)

In [6]:
#os.remove('weather_scrape.csv')

In [7]:
#os.remove('weather_scrape.xlsx')

In [6]:
date_time = datetime.now()

In [24]:
print(f'{date_time.day} : {date_time.month} : {date_time.year}')

27 : 3 : 2022


In [10]:
date_time.time()

datetime.time(17, 46, 19, 339134)